In [1]:
from multiprocessing import Pool, cpu_count
import pandas as pd
import numpy as np
from sklearn import *
import gc; gc.enable()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [2]:
dtypes = {} #Get Optimal
train = pd.read_csv('../data/train.csv', dtype=dtypes)
test = pd.read_csv('../data/test.csv', dtype=dtypes)
# zips = ['train_jpg.zip','test_jpg.zip']

## User Spending Pattern by using active data

In [4]:
train_a = pd.read_csv('../data/train_active.csv', usecols=['user_id','price'], dtype=dtypes)
train_a = train_a.groupby('user_id')['price'].agg(['mean','count']); gc.collect()
test_a = pd.read_csv('../data/test.csv', usecols=['user_id','price'], dtype=dtypes)
test_a = test_a.groupby('user_id')['price'].agg(['mean','count']); gc.collect()

14

In [5]:
train_a = train_a.reset_index()
train = pd.merge(train, train_a, how='left', on='user_id')
del train_a; gc.collect()
test_a = test_a.reset_index()
test = pd.merge(test, test_a, how='left', on='user_id')
del test_a; gc.collect()

35

In [6]:
train_user = train.groupby('user_id')['deal_probability'].agg(['mean']); gc.collect()
train_user = train_user.reset_index()
train = pd.merge(train, train_user, how='left', on='user_id')
test = pd.merge(test, train_user, how='left', on='user_id')
del train_user; gc.collect()

70

In [7]:
train.drop(columns=['item_id','user_id'], inplace=True)
train['image'] = train['image'].map(lambda x: 1 if len(str(x))>0 else 0)
test['image'] = test['image'].map(lambda x: 1 if len(str(x))>0 else 0)

In [8]:
test.drop(columns=['user_id'], inplace=True); gc.collect()

14

In [9]:
#Lazy for now
train['activation_date'] = pd.to_datetime(train['activation_date']).dt.day
test['activation_date'] = pd.to_datetime(test['activation_date']).dt.day

In [10]:
cat_cols = ['region', 'city', 'parent_category_name', 'category_name', 'param_1',
       'param_2', 'param_3', 'title', 'description',  'user_type']
for c in cat_cols:
    if c in ['title','description']:
        train[c + '_len'] = train[c].map(lambda x: len(str(x))) #Lenth
        train[c + '_wc'] = train[c].map(lambda x: len(str(x).split(' '))) #Word Count

        test[c + '_len'] = test[c].map(lambda x: len(str(x))) #Lenth
        test[c + '_wc'] = test[c].map(lambda x: len(str(x).split(' '))) #Word Count
        if c != 'description':
            feature_cnt = 20
            tfidf = feature_extraction.text.TfidfVectorizer(max_features=feature_cnt)
            train_tfidf = pd.DataFrame(tfidf.fit_transform(train[c].astype(str)).toarray())
            test_tfidf = pd.DataFrame(tfidf.transform(test[c].astype(str)).toarray())
            cols = train_tfidf.columns
            for i in range(feature_cnt):
                train[c + '_tfidf_' + str(i)] = train_tfidf[cols[i]]
                test[c + '_tfidf_' + str(i)] = test_tfidf[cols[i]]
    if c != 'description':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[c].unique())+list(test[c].unique()))
        train[c] = lbl.transform(train[c].astype(str))
        test[c] = lbl.transform(test[c].astype(str))
    print(c)

region
city
parent_category_name
category_name
param_1
param_2
param_3
title
description
user_type


In [11]:
train.fillna(-1, inplace=True)
test.fillna(-1, inplace=True)

In [12]:
import lightgbm as lgb

ex_col = ['item_id','user_id','deal_probability','description','mean_y']
col = [c for c in train.columns if c not in ex_col]

x1, x2, y1, y2 = model_selection.train_test_split(train[col], train.deal_probability.values, test_size=0.20, random_state=19)
#https://www.kaggle.com/shujian/avito-lightgbm-starter
params = {'learning_rate': 0.05, 'max_depth': 7, 'boosting': 'gbdt', 'objective': 'regression', 'metric': ['auc','rmse'], 'is_training_metric': True, 'seed': 19, 'num_leaves': 128, 'feature_fraction': 0.9, 'bagging_fraction': 0.8, 'bagging_freq': 5}
model2 = lgb.train(params, lgb.Dataset(x1, label=y1), 3000, lgb.Dataset(x2, label=y2), verbose_eval=200, early_stopping_rounds=100)
test['deal_probability'] = model2.predict(test[col], num_iteration=model2.best_iteration)
test['deal_probability'] = test['deal_probability'].clip(0., 1.)
test[['item_id','deal_probability']].to_csv('submission.csv', index=False)

Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.766321	valid_0's rmse: 0.229029
[400]	valid_0's auc: 0.769675	valid_0's rmse: 0.226851
[600]	valid_0's auc: 0.771806	valid_0's rmse: 0.225755
[800]	valid_0's auc: 0.772713	valid_0's rmse: 0.22509
[1000]	valid_0's auc: 0.773153	valid_0's rmse: 0.224642
[1200]	valid_0's auc: 0.77366	valid_0's rmse: 0.224279
[1400]	valid_0's auc: 0.77391	valid_0's rmse: 0.22402
Early stopping, best iteration is:
[1433]	valid_0's auc: 0.773952	valid_0's rmse: 0.223985


# 0.231 LB